In [1]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 467.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=1ac2a980e8810de0dac1916720a0fe2bf2a37bd324ec04232d4ee213548536d1
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas_ta


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mutual-funds-and-etfs/ETFs.csv
/kaggle/input/mutual-funds-and-etfs/MutualFund prices - F-K.csv
/kaggle/input/mutual-funds-and-etfs/MutualFund prices - L-P.csv
/kaggle/input/mutual-funds-and-etfs/MutualFund prices - A-E.csv
/kaggle/input/mutual-funds-and-etfs/ETF prices.csv
/kaggle/input/mutual-funds-and-etfs/MutualFunds.csv
/kaggle/input/mutual-funds-and-etfs/MutualFund prices - Q-Z.csv


In [4]:
ETF_stock = pd.read_csv("/kaggle/input/mutual-funds-and-etfs/ETF prices.csv").set_index("fund_symbol")
ETF_info = pd.read_csv("/kaggle/input/mutual-funds-and-etfs/ETFs.csv").set_index("fund_symbol")

In [5]:
def main_information(filename: str):
    df = pd.read_csv(filename)
    print(f'The Dataset {filename} contains {df.shape[0]} rows and {df.shape[1]} columns')
    print(
        f"There are {len(list(df.select_dtypes(exclude='number').columns))} categorical columns and  {len(list(df.select_dtypes(include='number').columns))} numerical columns")
    print(f'There are {len(list(df.columns[df.isnull().any()]))} columns with missing values')
    print(
        f'The maximum number of missing values is {df.isnull().sum().max()} which is representing {round((df.isnull().sum().max() / df.shape[0]) * 100, 3)} percent of the column')


In [6]:
def worldwide_description(data: pd.DataFrame):
    df = data[["fund_family", "fund_category"]].dropna()
    df = df.groupby(["fund_family","fund_category"]).size().reset_index().rename(columns={0: 'count'})
    fig = px.bar(df, x="fund_family", y="count", color="fund_category", text_auto=True, title="Fund category per fund family")
    fig.show()

In [7]:
def filter_by_size(data: pd.DataFrame):
    df = data[["size_type", "investment_type", "exchange_name"]].dropna()
    df = df.groupby(["size_type", "investment_type", "exchange_name"]).size().reset_index().rename(columns={0: 'count'})
    fig = px.bar(df, x="exchange_name", y="count", color="investment_type", facet_col="size_type", text_auto=True,
                 title="Type of investment depending on the size of the fund owner of the ETF")
    fig.show()

In [8]:
def fund_composition_sector(data: pd.DataFrame, fund_code:str):
    composition_col = data.iloc[:, 33:44]
    df = composition_col.loc[fund_code].T.reset_index()[:-1]
    df.columns = ["cat", "value"]
    df["cat"] = df["cat"].apply(lambda x: x.split("fund_sector_")[1])
    fig = px.pie(df, values="value", names="cat", title=f"Categories repartition for fund {fund_code}")
    fig.show()

In [9]:
def fund_composition_bond(data: pd.DataFrame, fund_code:str):
    composition_col = data.iloc[:, 48:59].dropna(subset=ETF_info.columns[51:59])
    df = composition_col.loc[fund_code].T.reset_index()[:-1]
    df.columns = ["cat", "value"]
    df["cat"] = df["cat"].apply(lambda x: x.split("fund_")[1])
    if len(df.columns) == 2:
        df.columns = ["cat", "value"]
        #df["cat"] = df["cat"].apply(lambda x: x.split("fund_sector_")[1])
        fig = px.pie(df[2:], values="value", names="cat", title=f"Bond repartition for fund {fund_code} - "
                                                                f"Bond Duration: {df.iloc[1,1]} - Bond Maturity: {df.iloc[0,1]}"
                                                                 f"- Percent of bond from US Government: {df.iloc[2,1]}")
        fig.show()
    else:
        print("No value or no bond in the ETF")

In [10]:
def fund_fundamental_ratio(data: pd.DataFrame, fund_code:str):
    df_ratio = data.iloc[ETF_info.index.get_loc(fund_code), 44:48]
    ratio = sorted([(i.split("fund_price_")[1]) for i in df_ratio.index])
    fig = go.Figure(data=[go.Table(
        header=dict(values=ratio,
                    fill_color='paleturquoise',
                    align='center'),
        cells=dict(values=df_ratio.values,
                   fill_color=['lavender']*4,
                   align='center'))
    ])
    fig.update_layout(
        title=f"Several Ratio for ETF {fund_code}"
    )
    fig.show()

In [11]:
def fund_performance(data: pd.DataFrame, fund_code: str):
    fund_perf = data.loc[fund_code].iloc[120:]
    perf_indicators = sorted(list(set(["_".join(i.split("fund_")[1].split("_")[:-1]) for i in fund_perf.index[:-1]])))
    perf_years = sorted(list(set([i.split("fund_")[1].split("_")[-1] for i in fund_perf.index[:-1]])),
                        key=lambda x: int(x.split("years")[0]))

    to_plot = pd.DataFrame(
        data=[
            [fund_perf.loc[f"fund_{j}_{i}"] for j in perf_indicators] for i in perf_years
        ],
        index=perf_years,
        columns=perf_indicators
    ).reset_index().rename(columns={"index": "year"})

    fig = go.Figure(data=[go.Table(
        header=dict(values=list(to_plot.columns),
                    fill_color='paleturquoise',
                    align='center'),
        cells=dict(values=[to_plot[i] for i in to_plot.columns],
                   fill_color=['lavender']+["white"]*(len(to_plot.columns)-1),
                   align='center'))
    ])
    fig.update_layout(
        title=f"ETF performance ratio over time for fund {fund_code}"
    )

    fig.show()

In [12]:
    def fund_evolution(data: pd.DataFrame, fund_code:str, kind:str):
        fund_perf = data.loc[fund_code]
        insight_list = ["SMA30", "SMA60", "SMA90", "CMA30", "EWMA30"]
        fund_perf.loc[:,"SMA30"] = fund_perf['adj_close'].rolling(30).mean()
        fund_perf.loc[:,"SMA60"] = fund_perf['adj_close'].rolling(60).mean()
        fund_perf.loc[:,"SMA90"] = fund_perf['adj_close'].rolling(90).mean()
        fund_perf.loc[:,"CMA30"] = fund_perf['adj_close'].expanding().mean()
        fund_perf.loc[:,"EWMA30"] = fund_perf['adj_close'].ewm(span=30).mean()
        fund_perf.ta.macd(close='adj_close', fast=12, slow=26, signal=9, append=True)

        fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                            vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'),
                            row_width=[0.2, 0.7], specs=[[{"secondary_y": True}], [{"secondary_y": True}]])

        if kind == "candle":
            fig.add_trace(go.Candlestick(name='Closing Price',
                                         x=fund_perf['price_date'],
                                         open=fund_perf['open'],
                                         high=fund_perf['high'],
                                         low=fund_perf['low'],
                                         close=fund_perf['close']), row=1, col=1)
        else:
            fig.add_trace(go.Scatter(
                    name='Closing Price',
                    x=fund_perf['price_date'],
                    y=fund_perf['adj_close']
                ), row=1, col=1)

        for insight in insight_list:
            fig.add_trace(go.Scatter(
                name=insight,
                x=fund_perf["price_date"],
                y=fund_perf[insight]
            ), row=1, col=1)

        fig.add_trace(go.Scatter(
            name="MACD_12_26_9",
            x=fund_perf["price_date"],
            y=fund_perf["MACD_12_26_9"],
        ), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Bar(x=fund_perf["price_date"], y=fund_perf['volume'], showlegend=False), row=2, col=1,secondary_y=False)
        fig.update(layout_xaxis_rangeslider_visible=False)
        fig.update_layout(
            yaxis_title='Wind speed (m/s)',
            title='Continuous, variable value error bars')
        fig.show()


In [13]:
main_information("/kaggle/input/mutual-funds-and-etfs/ETF prices.csv")
main_information("/kaggle/input/mutual-funds-and-etfs/ETFs.csv")

The Dataset /kaggle/input/mutual-funds-and-etfs/ETF prices.csv contains 3866030 rows and 8 columns
There are 2 categorical columns and  6 numerical columns
There are 0 columns with missing values
The maximum number of missing values is 0 which is representing 0.0 percent of the column
The Dataset /kaggle/input/mutual-funds-and-etfs/ETFs.csv contains 2310 rows and 142 columns
There are 17 categorical columns and  125 numerical columns
There are 129 columns with missing values
The maximum number of missing values is 2310 which is representing 100.0 percent of the column


In [14]:
worldwide_description(ETF_info)

In [15]:
filter_by_size(ETF_info)


In [16]:
fund_composition_sector(ETF_info, "AADR")


In [17]:
fund_composition_bond(ETF_info, "TFI")


In [18]:
fund_fundamental_ratio(ETF_info, "AADR")

In [19]:
fund_performance(ETF_info, "AADR")

In [20]:
fund_evolution(ETF_stock, "AADR", "candle")

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/pandas_ta/core.py:420: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

